
## what do I need to do?
- big map little map?
- 

# Exploration

In [1]:
!conda env list

# conda environments:
#
fst                   *  /Users/josephmann/anaconda3/envs/fst
root                     /Users/josephmann/anaconda3



In [2]:
import sys

sys.path.insert(0,'/Users/josephmann/Documents/GitHub/dask/dask/')

In [3]:
import numpy
import jmespath as jp
import pandas as pd
import re
from pymongo import MongoClient


In [4]:
from clean_osm import *
from importlib import reload

In [5]:
import clean_osm
reload(clean_osm)

<module 'clean_osm' from '/Users/josephmann/Documents/Notebooks/u2/clean_osm/__init__.py'>

In [6]:
reload(clean_osm.cl)

<module 'clean_osm.cl' from '/Users/josephmann/Documents/Notebooks/u2/clean_osm/cl.py'>

In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
from clean_osm.cl import *

## Set up test suite

In [9]:
import pytest


In [10]:
# !rm sample*
pytest.main(['-v','clean_osm'])

================================================================= test session starts =================================================================
platform darwin -- Python 3.6.1, pytest-3.1.2, py-1.4.34, pluggy-0.4.0 -- /Users/josephmann/anaconda3/envs/fst/bin/python
cachedir: .cache
rootdir: /Users/josephmann/Documents/Notebooks/u2, inifile:
collecting ... collected 5 items

clean_osm/test_clean.py::test_download_data PASSED
clean_osm/test_clean.py::test_get_element PASSED
clean_osm/test_clean.py::test_element2dict PASSED
clean_osm/test_clean.py::test_pf PASSED
clean_osm/test_clean.py::test_top_value_freqs PASSED

============================================================== 5 passed in 34.11 seconds ==============================================================


0

## Create bag

In [11]:
b1 = cl.pf()

partition files still fresh


## tag frequencies

In [12]:
b1.npartitions

8

In [13]:
b1.take(1)

({'attr': {'changeset': '43379308',
   'id': '29811182',
   'lat': '53.5354110',
   'lon': '-113.5079960',
   'timestamp': '2016-11-03T14:01:28Z',
   'uid': '1943392',
   'user': 'LogicalViolinist',
   'version': '27'},
  'tag': {'admin_level': '8',
   'capital': '4',
   'ele': '645',
   'is_in': {'continent': 'North America',
    'country': 'Canada',
    'country_code': 'CA',
    'iso_3166_2': 'CA-AB',
    'root': 'Alberta; Canada',
    'state': 'Alberta',
    'state_code': 'AB'},
   'name': {'ar': 'إدمونتون',
    'de': 'Edmonton',
    'en': 'Edmonton',
    'fr': 'Edmonton',
    'root': 'Edmonton',
    'ru': 'Эдмонтон',
    'uk': 'Едмонтон'},
   'place': 'city',
   'population': '812201',
   'state_capital': 'yes',
   'wikidata': 'Q2096',
   'wikipedia': 'en:Edmonton'},
  'type': 'node'},)

In [14]:
b1.map(lambda d: jp.search('keys(@)[]', d)).concat().frequencies().compute()

/Users/josephmann/anaconda3/envs/fst/lib/python3.6/site-packages/dask/bag/core.py:1063: UserWarning: Deprecated.  Use the .flatten method instead
  warn("Deprecated.  Use the .flatten method instead")


[('type', 422256), ('attr', 422256), ('tag', 422256)]

### `top_value_freqs`

In [15]:
def top_value_freqs(subtag, b1=b1, n=5):
    """
    parameters:
    ----------
    subtag (string): returns the top n frequencies of values
    for the key "subtag"
    if the value of "tag.[subtag]" is a dictionary then it returns 
    the top frequencies of the keys of  that dictionary.

    b1 (dask Bag): the bag that will be searched

    n (int): number of the most freqent tags and their frequencies to return

    returns:
    -------
    a dask bag of pairs (value, count)
    """

    expression = jp.compile(subtag)

    # make bag of subelements, remove None elements
    b2 = b1.map(lambda d: expression.search(d)).filter(lambda d: d)

    # make bag of lists comprised of key lists or just single element lists
    b3 = b2.map(lambda d: list(d.keys()) if isinstance(
        d, dict) else (d if isinstance(d, list) else [d]))

    # concatenate all lists together
    b4 = b3.flatten()

    # remove None elemens
    b5 = b4.filter(lambda d: d)

    # return bag with top n frequencies
    return b5.frequencies().topk(n, lambda x: x[1])

In [16]:
{k:w for k,w in cl.top_value_freqs('type', b1, n=20).compute()}

{'node': 366139, 'relation': 399, 'way': 55718}

In [89]:
top_value_freqs('attr.user', b1=b1, n=20).compute()

[('edmontongeo', 331481),
 ('xixi', 18233),
 ('Sundance', 14029),
 ('Feldo', 8225),
 ('yegbin', 7615),
 ('Mesowhite', 7578),
 ('ij_', 4759),
 ('jdkrabbe', 4658),
 ('ptrchpmn', 3110),
 ('ec90', 2000),
 ('AKC', 1826),
 ('Viajero Perdido', 1762),
 ('xelloss', 1740),
 ('yoltia', 1405),
 ('flight780', 1204),
 ('alester', 1063),
 ('ionutr_telenav', 931),
 ('Ranek', 922),
 ('ntrussell', 917),
 ('spince', 603)]

In [18]:
pd.DataFrame.from_records(top_value_freqs('attr', b1, n=10).compute())

,0,1
0,changeset,422256
1,id,422256
2,timestamp,422256
3,uid,422256
4,user,422256
5,version,422256
6,lat,366139
7,lon,366139


In [19]:
import dask
dask.__version__

'0.15.0'

In [20]:
%load_ext watermark

In [21]:
%watermark

2017-06-21T15:04:10-06:00

CPython 3.6.1
IPython 5.4.1

compiler   : GCC 4.2.1 Compatible Apple LLVM 6.1.0 (clang-602.0.53)
system     : Darwin
release    : 16.6.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit


In [22]:
import sys

In [23]:
pd.DataFrame.from_records(top_value_freqs('type', b1, n=10).compute())

,0,1
0,node,366139
1,way,55718
2,relation,399


In [24]:
pd.DataFrame.from_records(top_value_freqs('tag', b1, n=10).compute())

,0,1
0,building,41773
1,source,21279
2,addr,19262
3,highway,7300
4,name,4875
5,service,1599
6,amenity,1525
7,lanes,1229
8,oneway,1116
9,landuse,986


In [25]:
pd.DataFrame.from_records(top_value_freqs('tag.is_in', b1).compute())

,0,1
0,"Edmonton,Alberta",396
1,Edmonton,7
2,continent,2
3,country,2
4,country_code,2


# Data Cleaning

## Tale of Two Cities

In [26]:
b1.map(
    lambda d: jp.search('tag.addr.city',d)).distinct().compute()


['Edmonton', None, 'edmonton']

In [27]:
top_value_freqs('tag.addr.city',b1).compute()

[('Edmonton', 13057), ('edmonton', 1)]

In [28]:

def fix_city(doc):
    path = 'tag.addr.city'
    cpath = jp.compile(path)
    value = cpath.search(doc)
    if value:
        doc['tag']['addr']['city'] = 'Edmonton'

    return doc

In [29]:
top_value_freqs('addr.city', b1= b1.map(fix_city)).compute()

[]

## Five Provinces

In [30]:
b1.map(lambda d: jp.search('tag.addr.province', d)).filter(lambda d: d).distinct().compute()

['Aberta', 'Alberta', 'Alberta;AB', 'AVB', 'AB']

In [31]:
b1.map(lambda d: jp.search('tag.addr.state', d)).filter(lambda d: d).distinct().compute()

['AB']

### Provinces Fix

Previously we had the following distribution of entries for *province*.

In [32]:
pd.DataFrame(
    top_value_freqs('tag.addr.province',b1=b1).compute())

,0,1
0,AB,141
1,Alberta,29
2,AVB,1
3,Aberta,1
4,Alberta;AB,1


We change all entries with province to 'Alberta', independently of previous content.

In [33]:

def fix_province(doc):
    path = 'tag.addr.province'
    cpath = jp.compile(path)
    value = cpath.search(doc)
    if value:
        doc['tag']['addr']['province'] = 'Alberta'

    return doc

In [34]:
b1.map(fix_province).map(lambda d: jp.search('tag.addr.province', d)).filter(
    lambda d: d).distinct().compute()

['Alberta']

In [35]:
pd.DataFrame(
    top_value_freqs('tag.addr.province', b1=b1.map(fix_province)).compute())

,0,1
0,Alberta,173


After our fix we have the following:

## Non-standard postal codes

In [36]:
postal_re = re.compile(r'[A-Z]\d[A-Z] \d[A-Z]\d')
b1.map(lambda d: jp.search('tag.addr.postcode', d)).filter(
    lambda d: d).filter(lambda c: not postal_re.match(c)).compute()

['T5J-1J4',
 'T6e 2g9',
 'T5L4G4',
 'T5C3C8',
 'T6E1Z2',
 'T6C1N8',
 'T5J1E3',
 'AB T5J',
 'T5J1A7',
 'Alberta T6G',
 't6e 2g9',
 'AB T6E4S6',
 'T5J4B7',
 'T5H3J7',
 'T5K2X4',
 'T5N2N8',
 'T6G1C9',
 'T6E',
 'T5M0J2',
 'T5K0G1',
 'T5H2Z4',
 'T5H1A8',
 'T6E3Z5',
 'T6E3Z4',
 'T5M2P2',
 'T5J-0R2',
 'T5M0S7',
 'T6E',
 'T5N2T3',
 'T5R0L9',
 'T5N3L5',
 'T5N1H6',
 'T6A3X4',
 'T5N0N7',
 'T5N2V3',
 'T6C1W5',
 'T5M0V4',
 'T5H0B1',
 'T5G2E7',
 'T5B0A1',
 'T5N0W4',
 'T6G1T5',
 'T5M1P8',
 'T5J',
 'T5J5A2',
 'T5G0A6',
 'T6G',
 'T6G',
 'T5J',
 'T5J',
 'T5G3K2',
 'T5H2L3',
 'T6C1B5',
 'T5J',
 'T6G0X4',
 'T5J',
 'T5J',
 'T5H',
 'T5K2V4',
 'T5R5B3',
 'T6C1N8',
 'T6A1B6']

### Fix postal codes

In [37]:

def fix_postal_codes(doc):
    path = 'tag.addr.postcode'
    pc_re = re.compile(r'([A-Z]\d[A-Z])[-\s]*(\d[A-Z]\d){0,1}', re.IGNORECASE)
    cpath = jp.compile(path)
    value = cpath.search(doc)
    if value:
        matches = pc_re.search(value)
        if matches:
            s = "{}".format(matches.group(1))
            if matches.group(2):
                s += " " + matches.group(2)
            s = s.upper()
            doc['tag']['addr']['postcode'] = s

    return doc

In [38]:
postal_re = re.compile(r'[A-Z]\d[A-Z]( \d[A-Z]\d){0,1}')
b1.map(fix_postal_codes).map(lambda d: jp.search('tag.addr.postcode', d)).filter(
    lambda d: d).filter(lambda c: not postal_re.match(c)).compute()

[]

In [39]:
clean_b = b1.map(fix_postal_codes).map(fix_city).map(fix_province).map(json.dumps)

In [40]:
clean_b.take(1)

('{"type": "node", "attr": {"changeset": "43379308", "id": "29811182", "lat": "53.5354110", "lon": "-113.5079960", "timestamp": "2016-11-03T14:01:28Z", "uid": "1943392", "user": "LogicalViolinist", "version": "27"}, "tag": {"admin_level": "8", "capital": "4", "ele": "645", "is_in": {"root": "Alberta; Canada", "continent": "North America", "country": "Canada", "country_code": "CA", "iso_3166_2": "CA-AB", "state": "Alberta", "state_code": "AB"}, "name": {"root": "Edmonton", "ar": "\\u0625\\u062f\\u0645\\u0648\\u0646\\u062a\\u0648\\u0646", "de": "Edmonton", "en": "Edmonton", "fr": "Edmonton", "ru": "\\u042d\\u0434\\u043c\\u043e\\u043d\\u0442\\u043e\\u043d", "uk": "\\u0415\\u0434\\u043c\\u043e\\u043d\\u0442\\u043e\\u043d"}, "place": "city", "population": "812201", "state_capital": "yes", "wikidata": "Q2096", "wikipedia": "en:Edmonton"}}',)

In [41]:
!ls -lh clean-*.json

-rw-r--r--@ 1 josephmann  staff    12M  5 Jun 17:32 clean-0.json
-rw-r--r--@ 1 josephmann  staff    14M  5 Jun 17:32 clean-1.json
-rw-r--r--@ 1 josephmann  staff    14M  5 Jun 17:32 clean-2.json
-rw-r--r--@ 1 josephmann  staff    14M  5 Jun 17:32 clean-3.json
-rw-r--r--@ 1 josephmann  staff    14M  5 Jun 17:32 clean-4.json
-rw-r--r--@ 1 josephmann  staff    13M  5 Jun 17:32 clean-5.json
-rw-r--r--@ 1 josephmann  staff   5.8M  5 Jun 17:32 clean-6.json
-rw-r--r--@ 1 josephmann  staff   4.2M  5 Jun 17:32 clean-7.json


## Amenity frequencies

In [42]:
top_value_freqs('tag.amenity', n=10).compute()

[('parking', 521),
 ('restaurant', 168),
 ('fast_food', 129),
 ('cafe', 84),
 ('bicycle_parking', 64),
 ('bench', 60),
 ('school', 54),
 ('bank', 36),
 ('fuel', 36),
 ('place_of_worship', 36)]

In [44]:
b1.filter(lambda d: jp.search(
        "tag.amenity",d) == 'restaurant').pluck(
    'tag').pluck(
    'name','Nameless').filter(lambda d: isinstance(d, dict)).compute()

[{'en': 'Viphalay', 'root': 'Vidaley'},
 {'en': 'Maxs', 'root': 'Maxs'},
 {'fr': '爱民顿', 'root': '爱民顿'}]

In [45]:
b1.filter(lambda d: jp.search("tag.amenity", d) == 'restaurant').pluck(
    'tag', 'NoTag').pluck(
    'name', 'Nameless').map(lambda d: d.get('root', 'NoRoot') if isinstance(d, dict) else d).frequencies().topk(10, lambda t: t[1]).compute()

[('Nameless', 8),
 ('Boston Pizza', 5),
 ("Ricky's All Day Grill", 2),
 ("Tony Roma's", 2),
 ('Urban Diner', 2),
 ('Chop Steakhouse', 2),
 ('Urban China', 1),
 ("Humpty's Family Restaurant", 1),
 ("Moxie's", 1),
 ("Chandler's The Tea Shoppe", 1)]

In [46]:
b1.filter(lambda d: jp.search("tag.amenity",d) == 'restaurant').pluck(
    'tag',default='NoTag').map(lambda d: d.get('name', "__No_Name__")).compute()

['Urban China',
 "Humpty's Family Restaurant",
 "Moxie's",
 "Ricky's All Day Grill",
 "Chandler's The Tea Shoppe",
 "Kamil's Turkish Cafe & Restaurant",
 'Lee Garden',
 "Tony Roma's",
 'Nha Trang Vietnamese Restaurant',
 'Pho Hoan Pasteur Fusion Noodle House',
 'Blue Plate Diner',
 'Mikado',
 'Delux Burger Bar',
 "That's Aroma! A Garlic Restaurant",
 "Vi's for Pies",
 'C&S Seafood Restaurant',
 'Century Palace Restaurant',
 'Double Happiness',
 'Mirama Dining & Lounge',
 'El Rancho',
 'Trang Tien',
 'Mucho Burrito',
 'High Level Diner',
 'RedBike',
 'All Happy Family Restaurant',
 'Kyoto Japanese Cuisine',
 'Phơbulous',
 'Royal Pizza',
 'Pagolac Restaurant',
 "Doan's",
 'Culina Restaurant',
 "B's Diner",
 'Upper Crust Cafe & Caterers',
 'New Asian Village',
 "Bird's Nest of Beijing Restaurant",
 "Denny's",
 "Joey Tomato's",
 'Old Szechuan Restaurant',
 'Vighalay Thai Laos',
 'The Wokkery Restaurant',
 "Uncle Albert's Family Restaurant",
 'Noodle Noodle Restaurant',
 'Coliseum Steak Rib

In [47]:
b1.filter(lambda d: jp.search("tag.amenity",d) == 'restaurant').pluck(
    'tag',default='NoTag').pluck(
    'name','__No_Name__').map(lambda d: d.get('root','__No_Root__') if isinstance(d, dict) else d).frequencies().topk(10, lambda t: t[1]).compute()

[('__No_Name__', 8),
 ('Boston Pizza', 5),
 ("Ricky's All Day Grill", 2),
 ("Tony Roma's", 2),
 ('Urban Diner', 2),
 ('Chop Steakhouse', 2),
 ('Urban China', 1),
 ("Humpty's Family Restaurant", 1),
 ("Moxie's", 1),
 ("Chandler's The Tea Shoppe", 1)]

In [48]:
b1.filter(lambda d: jp.search(
        "tag.amenity",d) == 'cafe').pluck(
    'tag').pluck(
    'name','__No_Name__').map(lambda d: d['root'] if isinstance(d, dict) else d).frequencies().topk(10,lambda x:x[1]).compute()

[('Second Cup', 10),
 ('Starbucks Coffee', 9),
 ('Starbucks', 6),
 ('Tim Hortons', 5),
 ('__No_Name__', 4),
 ('Transcend Coffee', 2),
 ('Orange Julius', 2),
 ('Good Earth', 2),
 ("Jeffrey's Café & Wine Bar", 1),
 ('The Second Cup', 1)]

# Mongodb 

In [88]:
!ls -lh clean-?.json


-rw-r--r--@ 1 josephmann  staff    12M  5 Jun 17:32 clean-0.json
-rw-r--r--@ 1 josephmann  staff    14M  5 Jun 17:32 clean-1.json
-rw-r--r--@ 1 josephmann  staff    14M  5 Jun 17:32 clean-2.json
-rw-r--r--@ 1 josephmann  staff    14M  5 Jun 17:32 clean-3.json
-rw-r--r--@ 1 josephmann  staff    14M  5 Jun 17:32 clean-4.json
-rw-r--r--@ 1 josephmann  staff    13M  5 Jun 17:32 clean-5.json
-rw-r--r--@ 1 josephmann  staff   5.8M  5 Jun 17:32 clean-6.json
-rw-r--r--@ 1 josephmann  staff   4.2M  5 Jun 17:32 clean-7.json


In [49]:
client = MongoClient('localhost:27017')
mdb = client['examples']

In [50]:
mdb.p33.drop()

In [51]:
%%bash
ls clean-?.json | while read col; do
    mongoimport --db examples --collection p33 -j 8 --type json --file $col;
done

2017-06-21T15:20:00.529-0600	connected to: localhost
2017-06-21T15:20:01.380-0600	imported 52125 documents
2017-06-21T15:20:01.400-0600	connected to: localhost
2017-06-21T15:20:02.309-0600	imported 64857 documents
2017-06-21T15:20:02.328-0600	connected to: localhost
2017-06-21T15:20:03.227-0600	imported 64723 documents
2017-06-21T15:20:03.245-0600	connected to: localhost
2017-06-21T15:20:04.163-0600	imported 64704 documents
2017-06-21T15:20:04.183-0600	connected to: localhost
2017-06-21T15:20:05.114-0600	imported 64712 documents
2017-06-21T15:20:05.133-0600	connected to: localhost
2017-06-21T15:20:06.002-0600	imported 59144 documents
2017-06-21T15:20:06.021-0600	connected to: localhost
2017-06-21T15:20:06.472-0600	imported 29966 documents
2017-06-21T15:20:06.494-0600	connected to: localhost
2017-06-21T15:20:06.833-0600	imported 22088 documents


In [52]:
mdb.p33.count({})

422319

In [53]:
list(mdb.p33.find({"tag.landuse": {"$exists": 1}},{"tag.landuse": 1,'_id':0}))


[{'tag': {'landuse': 'quarry'}},
 {'tag': {'landuse': 'recreation_ground'}},
 {'tag': {'landuse': 'recreation_ground'}},
 {'tag': {'landuse': 'retail'}},
 {'tag': {'landuse': 'retail'}},
 {'tag': {'landuse': 'cemetery'}},
 {'tag': {'landuse': 'commercial'}},
 {'tag': {'landuse': 'reservoir'}},
 {'tag': {'landuse': 'reservoir'}},
 {'tag': {'landuse': 'recreation_ground'}},
 {'tag': {'landuse': 'recreation_ground'}},
 {'tag': {'landuse': 'residential'}},
 {'tag': {'landuse': 'retail'}},
 {'tag': {'landuse': 'residential'}},
 {'tag': {'landuse': 'recreation_ground'}},
 {'tag': {'landuse': 'retail'}},
 {'tag': {'landuse': 'retail'}},
 {'tag': {'landuse': 'commercial'}},
 {'tag': {'landuse': 'commercial'}},
 {'tag': {'landuse': 'retail'}},
 {'tag': {'landuse': 'commercial'}},
 {'tag': {'landuse': 'retail'}},
 {'tag': {'landuse': 'commercial'}},
 {'tag': {'landuse': 'recreation_ground'}},
 {'tag': {'landuse': 'retail'}},
 {'tag': {'landuse': 'recreation_ground'}},
 {'tag': {'landuse': 'retai

In [54]:
list(
    mdb.p33.aggregate([{
        "$sortByCount": "$tag.landuse"
    }, {
        "$limit": 30
    }]))

[{'_id': None, 'count': 421333},
 {'_id': 'residential', 'count': 560},
 {'_id': 'grass', 'count': 179},
 {'_id': 'retail', 'count': 127},
 {'_id': 'commercial', 'count': 59},
 {'_id': 'forest', 'count': 14},
 {'_id': 'recreation_ground', 'count': 14},
 {'_id': 'construction', 'count': 9},
 {'_id': 'industrial', 'count': 4},
 {'_id': 'basin', 'count': 3},
 {'_id': 'cemetery', 'count': 3},
 {'_id': 'reservoir', 'count': 3},
 {'_id': 'traffic_island', 'count': 2},
 {'_id': 'allotments', 'count': 2},
 {'_id': 'military', 'count': 2},
 {'_id': 'public', 'count': 1},
 {'_id': 'brownfield', 'count': 1},
 {'_id': 'mixed', 'count': 1},
 {'_id': 'publicllotmentsllotments', 'count': 1},
 {'_id': 'quarry', 'count': 1}]

In [55]:
list(
    mdb.p33.aggregate([{
        "$sortByCount": { "$substrBytes": ["$tag.addr.postcode", 0, 3]}
    }, {
        "$limit": 30
    }]))

[{'_id': '', 'count': 421970},
 {'_id': 'T5J', 'count': 94},
 {'_id': 'T6E', 'count': 46},
 {'_id': 'T6G', 'count': 41},
 {'_id': 'T5M', 'count': 40},
 {'_id': 'T5N', 'count': 38},
 {'_id': 'T6C', 'count': 24},
 {'_id': 'T5H', 'count': 21},
 {'_id': 'T5K', 'count': 20},
 {'_id': 'T5G', 'count': 11},
 {'_id': 'T6A', 'count': 4},
 {'_id': 'T5R', 'count': 3},
 {'_id': 'T5B', 'count': 2},
 {'_id': 'T6Z', 'count': 1},
 {'_id': 'T6J', 'count': 1},
 {'_id': 'T5C', 'count': 1},
 {'_id': 'T5L', 'count': 1},
 {'_id': 'T6K', 'count': 1}]

In [56]:
list(
    mdb.p33.aggregate([{
        "$match": {
            "tag.landuse": {
                "$in": ["grass", "forest", "recreation_ground"]
            }
        }
    }, {
        "$sortByCount": "$tag"
    }, {
        "$limit": 30
    }]))

[{'_id': {'landuse': 'grass'}, 'count': 72},
 {'_id': {'golf': 'fairway', 'landuse': 'grass'}, 'count': 39},
 {'_id': {'golf': 'tee', 'landuse': 'grass'}, 'count': 18},
 {'_id': {'landuse': 'grass', 'type': 'multipolygon'}, 'count': 8},
 {'_id': {'landuse': 'forest', 'leaf_cycle': 'mixed', 'leaf_type': 'mixed'},
  'count': 7},
 {'_id': {'landuse': 'recreation_ground'}, 'count': 6},
 {'_id': {'landuse': 'forest'}, 'count': 4},
 {'_id': {'area': 'yes', 'landuse': 'grass'}, 'count': 3},
 {'_id': {'golf': 'green',
   'landuse': 'grass',
   'leisure': 'pitch',
   'name': '1',
   'sport': 'golf'},
  'count': 2},
 {'_id': {'golf': 'green',
   'landuse': 'grass',
   'leisure': 'pitch',
   'name': '7',
   'sport': 'golf'},
  'count': 2},
 {'_id': {'golf': 'green',
   'landuse': 'grass',
   'leisure': 'pitch',
   'name': '17',
   'sport': 'golf'},
  'count': 2},
 {'_id': {'golf': 'green',
   'landuse': 'grass',
   'leisure': 'pitch',
   'name': '15',
   'sport': 'golf'},
  'count': 2},
 {'_id': 

In [57]:
list(
    mdb.p33.aggregate([ {"$match": {"tag.landuse": "grass"}},  
                       {"$sortByCount" : "$attr.user"},
        {"$limit": 30}
    ]))

[{'_id': 'jdkrabbe', 'count': 95},
 {'_id': 'yegbin', 'count': 60},
 {'_id': 'yoltia', 'count': 12},
 {'_id': 'Mesowhite', 'count': 7},
 {'_id': 'Moham837', 'count': 4},
 {'_id': 'Maarten Deen', 'count': 1}]

In [75]:
# how do I get the number of green spaces sorted by postal code?

r =     mdb.p33.aggregate([
        # filter entries to those containing postcodes
        {"$match" : {"tag.addr.postcode": {"$exists" : 1}}},
        
        # group on user and postcode
        {"$group" : {
            "_id" : {"user": "$attr.user","postcode" : {"$substrBytes": ["$tag.addr.postcode", 0, 3]}
                     },
            "count" : {"$sum" : 1}
            
        }},
        # flatten output
        {"$project" : {"user": "$_id.user", "postcode": "$_id.postcode", "count":1, "_id":0}},
        # sort by count descending
        {"$sort": {"count": -1} },
        # limit output to 20 items
        {"$limit": 20}
    ])

_df = pd.DataFrame.from_records(r)

In [76]:
_df

,count,postcode,user
0,63,T5J,Mesowhite
1,27,T5M,ptrchpmn
2,17,T6E,yegbin
3,13,T6C,yegbin
4,11,T5J,Yottabyte
5,11,T6E,Mesowhite
6,11,T5N,edmontongeo
7,11,T5N,ptrchpmn
8,10,T6G,yegbin
9,10,T6G,Mesowhite


In [ ]:
_df.assign( row_sum = _df.sum(axis=1))

In [ ]:
_s = _df.sum()
_s.name = "col_sum"

_df.append( _s )

In [ ]:
_s = _df.sum()
_s.name = "col_sum"
_df.assign( row_sum = _df.sum(axis=1)).append(_s)

# Big(ger) Data

In [77]:
# min_lat, max_lat, min_lon, max_lon =  53.5164, 53.5718, -113.5742,-113.4485
min_lat=53.4271
min_lon=-113.666
max_lat=53.6495
max_lon=-113.34

map_url = 'http://overpass-api.de/api/map?bbox={0},{1},{2},{3}'.format(min_lon, min_lat, max_lon, max_lat)
cl.download_data(map_url, fn='bigmap.osm')

In [78]:
import os
os.path.getsize('bigmap.osm') / 1024**2 , os.path.getsize('map.osm') / 1024**2

(502.4631099700928, 78.97756671905518)

In [79]:
!ls -lh *map.osm 

-rw-r--r--@ 1 josephmann  staff   502M  6 Jun 17:23 bigmap.osm
-rw-r--r--@ 1 josephmann  staff    79M 14 Jun 15:30 map.osm


In [94]:
# create partitions for parallel processing
big_b = cl.pf(fn = 'bigmap.osm')

making fresh partition files 9


In [95]:
!ls *_partition_?.osm

bigmap_partition_0.osm bigmap_partition_3.osm bigmap_partition_6.osm
bigmap_partition_1.osm bigmap_partition_4.osm bigmap_partition_7.osm
bigmap_partition_2.osm bigmap_partition_5.osm


In [96]:
clean_b = big_b.map(fix_postal_codes).map(fix_city).map(fix_province).map(json.dumps)

In [97]:
top_value_freqs('attr.user', b1=b1, n=20).compute()

[('edmontongeo', 331481),
 ('xixi', 18233),
 ('Sundance', 14029),
 ('Feldo', 8225),
 ('yegbin', 7615),
 ('Mesowhite', 7578),
 ('ij_', 4759),
 ('jdkrabbe', 4658),
 ('ptrchpmn', 3110),
 ('ec90', 2000),
 ('AKC', 1826),
 ('Viajero Perdido', 1762),
 ('xelloss', 1740),
 ('yoltia', 1405),
 ('flight780', 1204),
 ('alester', 1063),
 ('ionutr_telenav', 931),
 ('Ranek', 922),
 ('ntrussell', 917),
 ('spince', 603)]

In [98]:
top_value_freqs('attr.user', b1=big_b, n=20).compute()

[('edmontongeo', 2142198),
 ('xixi', 112465),
 ('Mesowhite', 94015),
 ('Sundance', 73349),
 ('yegbin', 22367),
 ('Shawn Benbow', 21194),
 ('charrois', 15168),
 ('bogdanp_telenav', 14462),
 ('Oberaffe', 13987),
 ('xelloss', 13415),
 ('geobase_stevens', 11148),
 ('Feldo', 9753),
 ('Viajero Perdido', 9322),
 ('ionutr_telenav', 8525),
 ('ec90', 8097),
 ('GiJo', 7861),
 ('jdkrabbe', 5672),
 ('VE6SRV', 5602),
 ('ntrussell', 5518),
 ('ij_', 5201)]

In [99]:
clean_b.to_textfiles('big_sample-*.json')

In [100]:
!ls -lh big_sample*

-rw-r--r--@ 1 josephmann  staff    76M 21 Jun 16:21 big_sample-0.json
-rw-r--r--@ 1 josephmann  staff    84M 21 Jun 16:21 big_sample-1.json
-rw-r--r--@ 1 josephmann  staff    84M 21 Jun 16:21 big_sample-2.json
-rw-r--r--@ 1 josephmann  staff    84M 21 Jun 16:21 big_sample-3.json
-rw-r--r--@ 1 josephmann  staff    84M 21 Jun 16:21 big_sample-4.json
-rw-r--r--@ 1 josephmann  staff    84M 21 Jun 16:21 big_sample-5.json
-rw-r--r--@ 1 josephmann  staff    37M 21 Jun 16:21 big_sample-6.json
-rw-r--r--@ 1 josephmann  staff    35M 21 Jun 16:21 big_sample-7.json


In [84]:
mdb.p33.drop()

In [101]:
%%bash
ls big_sample-?.json | while read col; do
    mongoimport --db examples --collection big -j 8 --type json --file $col;
done

2017-06-21T16:23:47.843-0600	connected to: localhost
2017-06-21T16:23:50.839-0600	[#############...........] examples.big	44.2MB/75.9MB (58.2%)
2017-06-21T16:23:52.951-0600	[########################] examples.big	75.9MB/75.9MB (100.0%)
2017-06-21T16:23:52.951-0600	imported 325820 documents
2017-06-21T16:23:52.974-0600	connected to: localhost
2017-06-21T16:23:55.974-0600	[###########.............] examples.big	39.4MB/84.3MB (46.8%)
2017-06-21T16:23:58.972-0600	[######################..] examples.big	78.6MB/84.3MB (93.3%)
2017-06-21T16:23:59.425-0600	[########################] examples.big	84.3MB/84.3MB (100.0%)
2017-06-21T16:23:59.425-0600	imported 400272 documents
2017-06-21T16:23:59.446-0600	connected to: localhost
2017-06-21T16:24:02.444-0600	[##########..............] examples.big	37.9MB/84.3MB (44.9%)
2017-06-21T16:24:05.443-0600	[#####################...] examples.big	75.5MB/84.3MB (89.5%)
2017-06-21T16:24:06.118-0600	[########################] examples.big	84.3MB/84.3MB (100.0%)


In [102]:
mdb.big.count({}),mdb.big.count({'type': 'node'}),mdb.big.count({'type': 'way'})

(2692150, 2341856, 348079)

In [103]:
list(mdb.big.aggregate([{"$sortByCount":'$type'}]))

[{'_id': 'node', 'count': 2341856},
 {'_id': 'way', 'count': 348079},
 {'_id': 'relation', 'count': 2215}]

In [106]:
# how do I get the number of green spaces sorted by postal code?
list(
    mdb.p33.aggregate([
        # filter entries to those containing postcodes
        {"$match" : {"tag.addr.postcode": {"$exists" : 1}}},
        
        # group on user and postcode
        {"$group" : {
            "_id" : {"user": "$attr.user","postalcode" : {"$substrBytes": ["$tag.addr.postcode", 0, 3]}
                     },
            "count" : {"$sum" : 1}
            
        }},
        # flatten output
        {"$project" : {"user": "$_id.user", "postalcode": "$_id.postalcode", "count":1, "_id":0}},
        # sort by count descending
        {"$sort": {"count": -1} },
        # limit output to 20 items
        {"$limit": 40}
    ]))

[{'count': 63, 'postalcode': 'T5J', 'user': 'Mesowhite'},
 {'count': 27, 'postalcode': 'T5M', 'user': 'ptrchpmn'},
 {'count': 17, 'postalcode': 'T6E', 'user': 'yegbin'},
 {'count': 13, 'postalcode': 'T6C', 'user': 'yegbin'},
 {'count': 11, 'postalcode': 'T6E', 'user': 'Mesowhite'},
 {'count': 11, 'postalcode': 'T5N', 'user': 'edmontongeo'},
 {'count': 11, 'postalcode': 'T5J', 'user': 'Yottabyte'},
 {'count': 11, 'postalcode': 'T5N', 'user': 'ptrchpmn'},
 {'count': 10, 'postalcode': 'T6G', 'user': 'Mesowhite'},
 {'count': 10, 'postalcode': 'T6G', 'user': 'yegbin'},
 {'count': 9, 'postalcode': 'T5K', 'user': 'Mesowhite'},
 {'count': 9, 'postalcode': 'T5N', 'user': 'TristanA'},
 {'count': 7, 'postalcode': 'T5M', 'user': 'edmontongeo'},
 {'count': 6, 'postalcode': 'T6G', 'user': 'ptrchpmn'},
 {'count': 5, 'postalcode': 'T6C', 'user': 'Mesowhite'},
 {'count': 5, 'postalcode': 'T5H', 'user': 'Mesowhite'},
 {'count': 4, 'postalcode': 'T5H', 'user': 'edmontongeo'},
 {'count': 4, 'postalcode': 

In [105]:
# how do I get the number of green spaces sorted by postal code?
list(
    mdb.big.aggregate([
        # filter entries to those containing postcodes
        {"$match" : {"tag.addr.postcode": {"$exists" : 1}}},
        
        # group on user and postcode
        {"$group" : {
            "_id" : {"user": "$attr.user","postalcode" : {"$substrBytes": ["$tag.addr.postcode", 0, 3]}
                     },
            "count" : {"$sum" : 1}
            
        }},
        # flatten output
        {"$project" : {"user": "$_id.user", "postalcode": "$_id.postalcode", "count":1, "_id":0}},
        # sort by count descending
        {"$sort": {"count": -1} },
        # limit output to 20 items
        {"$limit": 40}
    ]))

[{'count': 129, 'postalcode': 'T6T', 'user': 'Mesowhite'},
 {'count': 65, 'postalcode': 'T5J', 'user': 'Mesowhite'},
 {'count': 45, 'postalcode': 'T6L', 'user': 'Mesowhite'},
 {'count': 29, 'postalcode': 'T5T', 'user': 'TristanA'},
 {'count': 27, 'postalcode': 'T6E', 'user': 'yegbin'},
 {'count': 27, 'postalcode': 'T5M', 'user': 'ptrchpmn'},
 {'count': 20, 'postalcode': 'T6K', 'user': 'Mesowhite'},
 {'count': 19, 'postalcode': 'T6B', 'user': 'yegbin'},
 {'count': 17, 'postalcode': 'T6L', 'user': 'TristanA'},
 {'count': 17, 'postalcode': 'T6E', 'user': 'TristanA'},
 {'count': 16, 'postalcode': 'T6C', 'user': 'yegbin'},
 {'count': 13, 'postalcode': 'T5X', 'user': 'yegbin'},
 {'count': 13, 'postalcode': 'T5Z', 'user': 'TristanA'},
 {'count': 13, 'postalcode': 'T6G', 'user': 'yegbin'},
 {'count': 12, 'postalcode': 'T6A', 'user': 'yegbin'},
 {'count': 11, 'postalcode': 'T5N', 'user': 'ptrchpmn'},
 {'count': 11, 'postalcode': 'T5N', 'user': 'edmontongeo'},
 {'count': 11, 'postalcode': 'T6J',